In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

age, lulc, fire = desired_lulc()
mask_1985 = False
history_interval = 5
aggregate_LU_types = True

## Land Use Land Cover

### Grouping land use types (if aggregation is desired)

- annual_crops = 39, 20, 40, 62, 41
- perennial_crops = 46, 47, 35, 48
- pasture = 15
- mosaic = 21
- forest plantation = 9


In [2]:

# Listing land use types that are NOT forest
if (aggregate_LU_types):
    
    LU_index = [10,20,30,40,50]

    # A flat list of pixel values to replace.
    from_list = [15, 46, 47, 35, 48, 39, 20, 40, 62, 41, 21, 9, 3]

    # A corresponding list of new values.
    to_list = [10, 20, 20, 20, 20, 30, 30, 30, 30, 30, 40, 50, 3]

    # for each forest age, mask the lulc of the year immediately preceding abandonment
    lulc_aggregated = ee.Image()
    for band in lulc.bandNames().getInfo():
        lu_yr = lulc.select(band)
        remapped_band = lu_yr.remap(from_list, to_list).rename(band)
        lulc_aggregated = lulc_aggregated.addBands(remapped_band)

    lulc = lulc_aggregated.slice(1).byte()
else:
    LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21]



### Restricting history to n years before abandonment

Since we only have land use history dating back to 1985, the data is censored. In order to restrict history to exclude pixels with possible long histories of anthropogenic use pre-1985 which could not be included in the analysis:

- Make a mask with regrowing pixels that were forested in 1985 (keep only the patches that were forested in 1985, then subsequently deforested, and regrew after that).

- Making a mask including only pixels that have a specified number of bands (hist_interval) between the first and the last occurrence of an anthropogenic pixel (desired extent of land use history to include)

In [8]:

# for each year, make a mask for all anthropogenic pixels, and multiply by the year to have the
# pixel value be the year of observation of that land use type.
non_forest_years = ee.Image()
for yr in range(1985, 2020):
    year = f"{yr}"
    lu_yr = lulc.select(year).neq(3).multiply(yr)
    non_forest_years = non_forest_years.addBands(lu_yr)

# find the minimum and maximum non-zero value per pixel across all bands
# (first and last year of anthropogenic land use)
first_anthro = non_forest_years.selfMask().reduce(ee.Reducer.min())
last_anthro = non_forest_years.reduce(ee.Reducer.max())
# get total of continuous years from the first and last observation of anthropogenic land use
anthro_gaps = last_anthro.subtract(first_anthro).add(1)

if mask_1985:
    # select pixels that were forested in 1985
    # and that have at least history_interval years of anthropogenic land use
    lulc = lulc.updateMask(lulc.select("1985").eq(3)).selfMask() \
    .updateMask(age.lt(35-history_interval)).updateMask(anthro_gaps.lte(history_interval))


### Total sum of years under each desired land use type

For each desired land use type, the total occurrences per pixel are summed

In [4]:
LU_sum = ee.Image()
for val in LU_index:
    lulc_val = lulc.eq(val)
    num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f"lulc_sum_{val}")
    LU_sum = LU_sum.addBands(num_cells)

# removes the first (empty) band
LU_sum = LU_sum.slice(1).byte()

### Last Land Use Type

Finds what was the last land use type observed before abandonment.

In [6]:
# for each forest age, mask the lulc of the year immediately preceding abandonment
last_LU = ee.Image()
for yr in config.range_1986_2019:
    year = f"{yr}"
    lu_yr = lulc.select(year)
    age_mask = age.eq(config.last_year - yr)
    last_LU = last_LU.addBands(lu_yr.updateMask(age_mask))

# remove first empty band
# merge all images by adding them up
last_LU = last_LU.slice(1).reduce(ee.Reducer.sum()).rename("last_LU").byte()

### Fallow period length

Finds the total number of years that land was left fallow (classified as forest before regrowth)

In [9]:
nat_cover = lulc.eq(3)
total_nat_cover = nat_cover.reduce(ee.Reducer.sum())
mat_for_years = anthro_gaps.subtract(history_interval).multiply(-1).rename("mat_for_years").byte()
fallow = total_nat_cover.subtract(36).subtract(mat_for_years).add(10).rename("fallow").byte()

nat_cover = fallow.addBands(mat_for_years)

## Fire

### Total number of fires per pixel

Each burned pixel contains the value of the land use type that burned (such as the desired_values vector specified in the land use section) - unburned pixels are zero.

There are instances of fires:
- after the first observation of anthropogenic land use
- before the first observation of anthropogenic land use (forest fire)
- after the last observation of anthropogenic land use (burn in regrowing forest)

*Note that fire has different transform than lulc, and the projections will be adjusted when exporting.*

### How many years ago was the last fire?

Make one image with 34 bands, each with the number of years from the instance of burn to 2020.
Each pixel is 0 if unburned, and "time since fire" in number of years if it's burned.

In [3]:
# age = ee.Image(f"{data_folder}/eu/secondary").select("age")

# fire = (ee.Image("projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1")
#         .select([f"burned_coverage_{year}" for year in config.range_1985_2019])
#         .byte()
#         .rename([str(year) for year in config.range_1985_2019])
#         .updateMask(age))

# fire = fire.gt(0).byte()
# fires = fire.reduce(ee.Reducer.sum()).byte().rename("fires")

# export_image(fires, "total_fires", scale = 30, folder = "eu")

In [28]:
# Making a fire mask with pixels of value 1 for burned pixels and 0 otherwise
fire = fire.gt(0)

# Establishing a specific range for land use history for analysis
if mask_1985:
    first_year_anthro = 1986 + history_interval
    years_regrowth_considered = range(first_year_anthro, 2020)
else:
    years_regrowth_considered = config.range_1985_2019

# Initialize images
num_fires_before_regrowth = ee.Image()
num_fires_after_regrowth = ee.Image()

# Calculate burn information and add yearly bands to the corresponding images
for yr in range(1985, 2020):
    yrs_ago = 2020 - yr
    age_mask = age.eq(yrs_ago).selfMask()
    
    if yr < 2019:
        # Add bands for after regrowth
        bands_after = fire.select([str(year) for year in range(yr + 1, 2020)]).updateMask(age_mask)
        num_fires_after_regrowth = num_fires_after_regrowth.addBands(
            bands_after.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}")
        )
    
    if yr > 1985:
        # Add bands for before regrowth
        bands_before = fire.select([str(year) for year in range(1985, yr)]).updateMask(age_mask)
        num_fires_before_regrowth = num_fires_before_regrowth.addBands(
            bands_before.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}")
        )

# Remove the first band (assuming it's an unwanted initial band)
num_fires_before_regrowth = num_fires_before_regrowth.slice(1).reduce(ee.Reducer.sum()).rename("num_fires_before_regrowth")
num_fires_after_regrowth = num_fires_after_regrowth.slice(1).reduce(ee.Reducer.sum()).rename("num_fires_after_regrowth")

# Combine the before and after regrowth fire counts
num_fires = num_fires_before_regrowth.addBands(num_fires_after_regrowth).byte()

# Unmask to fill any missing values with 0
num_fires = num_fires.unmask(0)

In [23]:
# Making a fire mask with pixels of value 1 for burned pixels and 0 otherwise instead:
fire = fire.gt(0)

"""
Establishing a specific range for land use history for analysis, 
the ages of forests regrowing are also restricted.
For example, if we have a fixed 10 years of history:
1985 ---------- 10 years ----------- 1995 ----------- 25 years -----------2020
the regrowing forests have started regrowing in 1995 at the very earliest.

Then, the years of regrowth considered are:
"""

if mask_1985 == True:
    first_year_anthro = 1986 + history_interval
    years_regrowth_considered = range((first_year_anthro), 2020)
else:
    years_regrowth_considered = config.range_1985_2019

# Initialize images
num_fires_before_regrowth = ee.Image()
num_fires_after_regrowth = ee.Image()

# Calculate burn information and add yearly bands to the corresponding images
for yr in range(1985, 2019):
    yrs_ago = 2020 - yr
    age_mask = age.eq(yrs_ago).selfMask()
    
    # Add bands for after regrowth
    bands_after = fire.select([str(year) for year in range(yr + 1, 2020)]).updateMask(age_mask)
    num_fires_after_regrowth = num_fires_after_regrowth.addBands(
        bands_after.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}")
    )

# Calculate burn information and add yearly bands to the corresponding images
for yr in range(1986, 2020):
    yrs_ago = 2020 - yr + 1
    age_mask = age.eq(yrs_ago).selfMask()
    
    # Add bands for before regrowth
    bands_before = fire.select([str(year) for year in range(1985, yr)]).updateMask(age_mask)
    num_fires_before_regrowth = num_fires_before_regrowth.addBands(
        bands_before.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}")
    )

# Remove the first band (assuming it's an unwanted initial band)
num_fires_before_regrowth = num_fires_before_regrowth.slice(1).reduce(ee.Reducer.sum()).rename("num_fires_before_regrowth")
num_fires_after_regrowth = num_fires_after_regrowth.slice(1).reduce(ee.Reducer.sum()).rename("num_fires_after_regrowth")


# Apply the helper function to each band
num_fires = num_fires_before_regrowth.addBands(num_fires_after_regrowth).byte()

num_fires = num_fires.unmask(0)

In [5]:
# yr = 1985
# yrs_ago = 2020 - yr
# age_mask = age.eq(yrs_ago).selfMask()

# bands_before = fire.select([str(year) for year in range(1985, yr)]).updateMask(age_mask)

# map = geemap.Map()
# map.addLayer(bands_before, {}, "bands_before")
# map.addLayer(age_mask, {}, "age_mask")
# map.addLayer(fire, {}, "fire")
# map

In [29]:
# yr = 2010
# yrs_ago = 2020 - yr
# age_mask = age.eq(yrs_ago)
# num_fires_after_regrowth

# num_fires_before_regrowth = ee.Image()
# bands = fire.select(
#     [f"{year}" for year in range(1985, 2010)]
# ).updateMask(age_mask)

# num_fires_before_regrowth = num_fires_before_regrowth.addBands(bands.reduce(ee.Reducer.sum())
#                                                              .byte().rename("2010"))
# num_fires_before_regrowth = num_fires_before_regrowth.slice(1)


map = geemap.Map()
# Add layers with transparency
map.addLayer(age, {"opacity": 0.1}, "age")
map.addLayer(num_fires.updateMask(num_fires_after_regrowth.gt(0)), {}, "num_fires")  # Assuming num_fires should not be transparent
map.addLayer(fire, {"opacity": 0.1}, "fire")
map.addLayer(lulc, {"opacity": 0.1}, "lulc")

map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Export land_use

In [11]:
land_use = LU_sum.addBands([last_LU, nat_cover, fires])

# name = ""
# if aggregate_LU_types:
#     name += "_aggregated"
# if mask_1985:
#     name += f"_{history_interval}_yr"

# export_image(land_use, f"land_use{name}", scale = 30)

In [12]:
unburned_mask = land_use.select("num_fires_after_regrowth").eq(0)

map = geemap.Map()
map.addLayer(unburned_mask, {}, "unburned_mask")
map.addLayer(lulc, {}, "lulc")
map.addLayer(lulc, {}, "fire")
map

EEException: Image.updateMask: If one image has no bands, the other must also have no bands. Got 0 and 1.